**Run Setup**

In [2]:
%run ./Setup.ipynb

**Search all (\*) Purview assets and create a csv file with asset details**

In [3]:
#you can change the keyword to match your scope
cnt, dfAssets = searchPurviewAssets(keyword='*', recursive_read=True)

#save the file to root working folder
export_file_name = os.path.join(root_working_folder, asset_export_file_name_template.format(os.getenv('PURVIEW_NAME')))

if os.path.exists(export_file_name):
    os.remove(export_file_name)

dfAssets.to_csv(export_file_name, index_label='index')

**Crawl assets and extract schema information  
This will be taking a longer time to complete.  
If you don't need schema information, you can skip this section**

In [4]:
columns_list = dfAssets.columns
dfDetiledAssets = pd.DataFrame(columns = columns_list)

for index, row in dfAssets.iterrows():
    dfDetiledAssets = dfDetiledAssets.append(row, ignore_index=True)

    thisAssetId = dfAssets.loc[index]['id']
    bulk_output = !pv entity readBulk --guid {thisAssetId}
    bulk_json = json.loads(''.join(bulk_output))
    if len(bulk_json['entities'])!=1:
        continue;

    cols, refEntities = getColumnStructure(bulk_json)

    if cols == None:
        continue;

    for aCol in cols:
        colId = aCol['guid']
        #find column in referenced entites section
        thisRefEntity = refEntities[colId]
        
        strClassifications = None
        strTerms = None
            
        if 'classifications' in thisRefEntity:
            strClassifications = separator_char.join([aClass['typeName'] for aClass in thisRefEntity['classifications']])

        if 'meanings' in thisRefEntity['relationshipAttributes']:
            thisTermsGuidList = [aTerm['guid'] for aTerm in thisRefEntity['relationshipAttributes']['meanings']]
            thisTermsList = guidToFormalName(thisTermsGuidList)  
            strTerms = separator_char.join(thisTermsList)


        this_col_row = {'name':thisRefEntity['attributes']['name']
        ,'qualifiedName':thisRefEntity['attributes']['qualifiedName']
        ,'classification':strClassifications
        ,'term':strTerms
        ,'description':thisRefEntity['attributes']['description']
        ,'entityType':thisRefEntity['typeName']
        ,'assetType':''
        ,'id':colId}

        dfDetiledAssets = dfDetiledAssets.append(this_col_row, ignore_index=True)
    
    
#save the file to root working folder
export_details_file_name = os.path.join(root_working_folder, asset_detail_export_file_name_template.format(os.getenv('PURVIEW_NAME')))
dfDetiledAssets.to_csv(export_details_file_name, index_label='index')

In [5]:
print(str(len(dfAssets)) + " asset(s) were extracted from " + os.getenv('PURVIEW_NAME') )
print(str(len(dfDetiledAssets)) + " asset(s) with details were extracted from " + os.getenv('PURVIEW_NAME') )